In [81]:
# coding: utf-8
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.pipeline import Pipeline

from sklearn.naive_bayes import MultinomialNB
from nltk import word_tokenize
from nltk.tokenize import wordpunct_tokenize

import os
import random

In [82]:
def plot_sigmoid():
	X = np.linspace(-6, 6, 200)
	Y = 1 / (1 + np.exp(-X))

	plt.plot(X, Y)
	plt.grid(True)
	plt.xlabel('X')
	plt.ylabel('logistic function')
	plt.show()

In [83]:
negative = [f for f in os.listdir('./review_polarity 2/txt_sentoken/neg') if f.endswith('.txt')]

In [84]:
positive = [f for f in os.listdir('./review_polarity 2/txt_sentoken/pos') if f.endswith('.txt')]

In [85]:
arr_reviews = []

for review in negative:
    with open('./review_polarity 2/txt_sentoken/neg/' + review, 'r', encoding='utf-8') as r:
        dicti = {}
        text = r.read()
        dicti['class'] = '0'
        dicti['text'] = text
        arr_reviews.append(dicti)
        
for review in positive:
    with open('./review_polarity 2/txt_sentoken/pos/' + review, 'r', encoding='utf-8') as r:
        dicti = {}
        text = r.read()
        dicti['class'] = '1'
        dicti['text'] = text
        arr_reviews.append(dicti)

        
random.shuffle(arr_reviews)  # перемешиваем данные
# for line in arr:
#     if line['class'] == '1':
#         print(line)
    
df = pd.DataFrame(arr_reviews)  # создаем датасет


In [86]:
df.head()

,class,text
0,0,michael robbins' hardball is quite the cinemat...
1,0,you should have heard the old guys in the crow...
2,0,lucas was wise to start his star wars trilogy ...
3,0,"set in harlem during the great depression , ri..."
4,1,"david lynch's "" blue velvet "" begins and ends ..."


In [87]:
df.describe()

,class,text
count,2000,2000
unique,2,2000
top,1,"at first glance , i thought that the sword and..."
freq,1000,1


In [88]:
df.groupby('class').describe()

text
class                                                          
0     count                                                1000
      unique                                               1000
      top     at first glance , i thought that the sword and...
      freq                                                    1
1     count                                                1000
      unique                                               1000
      top      " it's not good to know too much about someon...
      freq                                                    1

In [89]:
#  Делим выборку в соотновении 80:20
rev_train, rev_test, class_train, class_test = train_test_split(df['text'], df['class'], test_size=0.2)

In [90]:
rev_train.shape, rev_test.shape, class_train.shape, class_test.shape

((1600,), (400,), (1600,), (400,))

In [91]:
# Векторизация
bow = CountVectorizer()
bow.fit_transform(rev_train)
bowed_rev_train = bow.transform(rev_train)  # тренировочные тексты
bowed_rev_test = bow.transform(rev_test)   # тестовые тексты

In [92]:
# Обучаем модель логистической регрессии
clf = LogisticRegression(class_weight='balanced')
clf.fit(bowed_rev_train, class_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [93]:
print(classification_report(class_test, clf.predict(bowed_rev_test)))

             precision    recall  f1-score   support

          0       0.84      0.87      0.86       185
          1       0.89      0.86      0.87       215

avg / total       0.87      0.86      0.87       400



In [94]:
clf.coef_

array([[  1.09569519e-02,  -1.28542592e-02,   8.72086557e-06, ...,
          1.07465888e-03,   1.70542903e-03,  -2.48964022e-03]])

In [95]:
clf.intercept_

array([-0.54294908])

In [96]:
clf.classes_

array(['0', '1'], dtype=object)

In [98]:
df.hist()

ValueError: num must be 1 <= num <= 0, not 1

In [ ]:
class_train.hist()